In [1]:
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set() 
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, plot_tree, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, SVR


from datetime import datetime, timedelta
from alive_progress import alive_bar, alive_it 
import random


matches_dataset = pd.read_csv("all_matches_dataset.csv")
matches_dataset

PLAYER_STAT_VARIABLES = [var.replace("t1p1_","") for var in list(matches_dataset.columns) if ("t1p1_" in var) and (not "player" in var)]
print("All player stats variables:", ", ".join(PLAYER_STAT_VARIABLES))

many_pstat_vars = ["kills", "assists", "deaths", "kdratio", "kddiff", "adr", "fkdiff", "rating"]
few_pstat_vars = ["kills", "kdratio", "kddiff", "fkdiff", "rating"]
min_pstat_vars = ["kills", "kdratio", "rating"]

SELECTED_PSTAT_VARS = min_pstat_vars
print("SELECTED player stats variables:", ", ".join(SELECTED_PSTAT_VARS))

All player stats variables: kills, hskills, assists, fassists, deaths, kdratio, kddiff, adr, fkdiff, rating
SELECTED player stats variables: kills, kdratio, rating


In [2]:


def filter_dataset_by_date(full_df, start_dt : datetime, stop_dt : datetime, date_col_name : str = "match_date"):
    df : DataFrame = full_df.copy()
    keep_dict = {}
    if start_dt > stop_dt:
        print("datetimes were given backwards")
        temp_dt = start_dt
        start_dt = stop_dt
        stop_dt = temp_dt
    for index, row in df.iterrows():
        if not((dt:=datetime.strptime(row[date_col_name], '%Y-%m-%d')) < start_dt or dt >= stop_dt):
            # keep_indeces.append(index)
            # keep_rows.append(row)
            keep_dict[index] = row 
    return DataFrame.from_dict(keep_dict,orient="index")


def partition_dataset_by_date(df : DataFrame, stop_date = "2017-08-01", partition_length : int = 120, overlapping : bool = True, partition_shift : int = 30, pstat_vars_filter = None) -> Series:
    partitions = {}
    stop_dt = datetime.strptime(stop_date, '%Y-%m-%d')
    dt_it = datetime.now()
    if not overlapping:
        partition_shift = partition_length
    print("Partitioning dataset temporally (by date):")
    while dt_it - timedelta(days=(partition_length)) > stop_dt:
        partition = filter_dataset_by_date(df, date_col_name= "match_date", start_dt=dt_it - timedelta(days = partition_length), stop_dt=dt_it)
        date_range = f"{datetime.strftime(dt_it - timedelta(days = partition_length),'%Y-%m-%d')} -> {datetime.strftime(dt_it,'%Y-%m-%d')}"
        partitions[date_range] = partition
        print(date_range,":",partition.shape[0],"items in partition")
        dt_it = dt_it - timedelta(days=partition_shift)
    return Series(partitions)


def get_aggregate_players_stats(df : DataFrame, pstat_vars = SELECTED_PSTAT_VARS):
    teams_pstats = {}

    for index, row in df.iterrows():
        for tn in (1,2):
            # score_diff = row["t1_total_rw"] - row["t2_total_rw"] if tn==1 else row["t2_total_rw"] - row["t1_total_rw"]
            tname = row[f"t{tn}_name"]
            
            for pn in (1,2,3,4,5):
                id = f"t{tn}p{pn}_"
                pname = row[id+"player"]
                if teams_pstats.get(tname) == None:
                    teams_pstats[tname] = {}
                if teams_pstats[tname].get(pname) == None:
                    teams_pstats[tname][pname] = []
                teams_pstats[tname][pname].append({var:row[id+var] for var in pstat_vars})
    return teams_pstats


def get_active_rosters(matches_df) -> dict[str, list[str]]:
    aggregate_pstats = get_aggregate_players_stats(matches_df)
    teams_active_players = {}
    for tname in aggregate_pstats:
        team = aggregate_pstats[tname]
        players_games_played = {}
        # players_games_played =  {player: for player in team}
        for pname in team:
            player = team[pname]
            players_games_played[pname] = len(list(player))
        active_players = list(pd.Series(players_games_played).sort_values(ascending=False).iloc[:5].index)
        teams_active_players[tname] = active_players
    return teams_active_players


def get_sorted_teams_rosters(df : DataFrame, rosters, mode = "mean"):
    aggregated_teams_pstats = get_aggregate_players_stats(df)
    sorted_rosters = {}
    for tname in rosters:
        team = aggregated_teams_pstats[tname]
        pn = 0
        pratings = {}
        for pname in rosters[tname]:
            pmatches = pd.DataFrame(team[pname])
            prating_stats = pmatches.loc[:,["rating"]].describe().squeeze()
            prating_agg_val = prating_stats[mode]
            pratings[pname] = prating_agg_val
        
        sorted_rosters[tname] = list(Series(pratings).sort_values(ascending=False).index)
    return sorted_rosters

# active_rosters = get_active_rosters(recent_matches_dataset)


def make_teams_profiles(df : DataFrame, pstat_vars = SELECTED_PSTAT_VARS, mode = "mean"):
    
    rosters = get_sorted_teams_rosters(df, rosters=get_active_rosters(df))

    aggregated_teams_pstats = get_aggregate_players_stats(df, pstat_vars=pstat_vars)

    teams_profiles = {}
    for tname in rosters:
        team = aggregated_teams_pstats[tname]
        team_flattened_pstats = {}
        pn = 0
        for pname in rosters[tname]:
            pn += 1
            pmatches = pd.DataFrame(team[pname])
            pmatches_stats = pmatches.describe()
            pstats : Series = pmatches_stats.loc[[mode], :].squeeze()
            for index, value in pstats.iteritems():
                team_flattened_pstats[f"p{pn}_{index}"] = value
        teams_profiles[tname] = Series(team_flattened_pstats).astype("float32")
    return teams_profiles




-----
-----

# SETUP EVERYTHING 

-----

In [3]:

# import crawler, team_rankings_crawler_and_scraper, matches_stats_scraper


def remove_unused_pstats(df : DataFrame, keep_pstat_vars : list [str]) -> DataFrame:
    keep_cols = [var for var in list(df.columns) if not((x:=var.split("_")[-1]) in PLAYER_STAT_VARIABLES) or x in keep_pstat_vars] 
    return df.loc[:, keep_cols ]

matches_dataset = remove_unused_pstats(matches_dataset, SELECTED_PSTAT_VARS)
matches_dataset = matches_dataset.sort_values(by=["match_date"], ascending=False)
matches_dataset

dataset_partitions = partition_dataset_by_date(matches_dataset, partition_length=180, overlapping=False)
recent_matches_dataset = dataset_partitions.iloc[0]

Partitioning dataset temporally (by date):
2021-10-28 -> 2022-04-26 : 5385 items in partition
2021-05-01 -> 2021-10-28 : 6906 items in partition
2020-11-02 -> 2021-05-01 : 5563 items in partition
2020-05-06 -> 2020-11-02 : 6222 items in partition
2019-11-08 -> 2020-05-06 : 5169 items in partition
2019-05-12 -> 2019-11-08 : 6264 items in partition
2018-11-13 -> 2019-05-12 : 5846 items in partition
2018-05-17 -> 2018-11-13 : 7351 items in partition
2017-11-18 -> 2018-05-17 : 6679 items in partition


-----
-----

# WINNER PREDICTOR MODELS

-----

In [4]:
def convert_dataset_for_classification_models(df : DataFrame, seed : int = 1, test_set_size : float = 0.3):
    
    teams_profiles : dict[str,Series] = make_teams_profiles(df, pstat_vars=SELECTED_PSTAT_VARS)

    # convert all unranked teams to have rank of 31st position
    model_dataset = df.copy()
    model_dataset["t1_rank"].fillna(31, inplace=True)
    model_dataset["t2_rank"].fillna(31, inplace=True)

    # extract data for each team, extract player stats later
    teams_stats = model_dataset.loc[:,["t1_rank", "t2_rank", "t1_name", "t2_name"]]

    # add team player stat profiles to each entry / row of data in features dataframe
    features_dict = {}
    for index, row in teams_stats.iterrows():
        t1_name, t2_name = row["t1_name"], row["t2_name"]
        if t1_name in teams_profiles or t2_name in teams_profiles:
            if not t1_name in teams_profiles:
                t2_profile = Series(list(teams_profiles[t2_name].values), 
                    index = [f"t2{pstat_var}" for pstat_var in list(teams_profiles[t2_name].index)])
                t1_profile = Series(list(teams_profiles[t2_name].apply(lambda x: None).values), 
                    index = [f"t1{pstat_var}" for pstat_var in list(teams_profiles[t2_name].index)])
            elif not t2_name in teams_profiles:
                t1_profile = Series(list(teams_profiles[t1_name].values), 
                    index = [f"t1{pstat_var}" for pstat_var in list(teams_profiles[t1_name].index)])
                t2_profile = Series(list(teams_profiles[t1_name].apply(lambda x: None).values), 
                    index = [f"t2{pstat_var}" for pstat_var in list(teams_profiles[t1_name].index)])
            else:
                t1_profile = Series(list(teams_profiles[t1_name].values), 
                    index = [f"t1{pstat_var}" for pstat_var in list(teams_profiles[t1_name].index)])
                t2_profile = Series(list(teams_profiles[t2_name].values), 
                    index = [f"t2{pstat_var}" for pstat_var in list(teams_profiles[t2_name].index)])
            features_dict[index] = pd.concat([row, t1_profile, t2_profile])
            # keep_dict[index] = row
        # else:
            # model_dataset.drop(index, inplace=True)
    
    features = DataFrame.from_dict(features_dict, orient="index")
    features.drop(columns=["t1_name", "t2_name"], inplace=True)
    features.fillna(0,inplace=True)

    # numerical_vars = [var for var in list(features.columns) if var[0] == "t" and var[2] == "p"]
    numerical_vars = list(features.columns)
    
    labels = model_dataset["winner"].astype("category")
    # print(features.shape, labels.shape)
    xtrain, xtest, ytrain, ytest =  train_test_split( features, labels, random_state=seed, test_size=test_set_size)

    scaler = StandardScaler()
    scaler.fit(xtrain[numerical_vars])

    xtrain[numerical_vars] = scaler.transform(xtrain[numerical_vars]) #scale the training data
    xtest[numerical_vars] = scaler.transform(xtest[numerical_vars]) #scale the testing data

    return xtrain, xtest, ytrain, ytest


# todo : add feature importance visualiztions for tuning / selecting features to use in the models

In [5]:

def initial_test_winner_models():
    xtrain, xtest, ytrain, ytest = convert_dataset_for_classification_models(recent_matches_dataset ,seed=69)

    basic_rank_pred = []
    for index, row in xtest.iterrows():
        basic_rank_pred.append("t1" if row["t1_rank"] >= row["t2_rank"] else "t2")
    print(f'The baseline predictor has a prediction accuracy of {round(accuracy_score(ytest,basic_rank_pred)*100,2)}%')

    random.seed(69)
    rand_pred = []
    for index, row in xtest.iterrows():
        rand_pred.append("t1" if random.random() < 0.5 else "t2")
    print(f'The random predictor has a prediction accuracy of {round(accuracy_score(ytest,rand_pred)*100,2)}%')

    ratio_rank_pred = []
    for index, row in xtest.iterrows():
        ratio_rank_pred.append("t1" if random.random() > float(row["t1_rank"]) / float(row["t1_rank"] + row["t2_rank"]) else "t2")
    print(f'The ratio rank predictor has a prediction accuracy of {round(accuracy_score(ytest,ratio_rank_pred)*100,2)}%')

    knn = KNeighborsClassifier(n_neighbors=15)
    knn.fit(xtrain,ytrain)
    print(f'The KNeighborsClassifier model has a prediction accuracy of {round(knn.score(xtest, ytest)*100,2)}%')

    dtc = DecisionTreeClassifier(random_state=69)
    dtc.fit(xtrain, ytrain)
    print(f'The DecisionTreeClassifier model has a prediction accuracy of {round(dtc.score(xtest, ytest)*100,2)}%')

    rf = RandomForestClassifier(criterion='entropy', n_estimators = 100, random_state = 69)
    rf.fit(xtrain, ytrain)
    print(f'The RandomForestClassifier model has a prediction accuracy of {round(rf.score(xtest, ytest)*100,2)}%')
    
    nb = GaussianNB()
    nb.fit(xtrain, ytrain)
    print(f'The GaussianNB model has a prediction accuracy of {round(nb.score(xtest, ytest)*100,2)}%')

    svm = SVC()
    svm.fit(xtrain, ytrain)
    print(f'The SVC model has a prediction accuracy of {round(svm.score(xtest, ytest)*100,2)}%')
    

# initial_test_winner_models()

In [9]:

############### UNCOMMENT BELOW TO VIEW ALL TUNABLE PARAMS FOR THE VARIOUS MODELS ###############
# dummy_knn = KNeighborsClassifier()
# print("Available params for knn():")
# print(", ".join([f"{key}: {dummy_knn.get_params()[key]}" for key in dummy_knn.get_params()]))

# dummy_dtc = DecisionTreeClassifier()
# print("\nAvailable params for dtc():")
# print(", ".join([f"{key}: {dummy_dtc.get_params()[key]}" for key in dummy_dtc.get_params()]))

# dummy_rf = RandomForestClassifier()
# print("\nAvailable params for rf():")
# print(", ".join([f"{key}: {dummy_rf.get_params()[key]}" for key in dummy_rf.get_params()]))

# dummy_nb = GaussianNB()
# print("\nAvailable params for nb():")
# print(", ".join([f"{key}: {dummy_nb.get_params()[key]}" for key in dummy_nb.get_params()]))

# dummy_svm = SVC()
# print("\nAvailable params for svc():")
# print(", ".join([f"{key}: {dummy_svm.get_params()[key]}" for key in dummy_svm.get_params()]))


def tune_winner_models():

    knn_tuning_results = {}
    dtc_tuning_results =  {}
    rf_tuning_results = {}
    nb_tuning_results = {}
    svm_tuning_results = {}

    for daterange, part in dataset_partitions.iteritems():
        print(daterange)
        # 10 different seeds for us to average results across
        # random_states = [6, 13, 23, 24, 33, 69, 420, 666, 6969, 13666]
        # random_states = [69, 420, 666]
        random_states = [n + n**n for n in range(0,5)]
        # knn_tuning_results_dict =  {}
        xtrain, xtest, ytrain, ytest = convert_dataset_for_classification_models(df=part, seed=1)

        result = {}
        # xtrain, xtest, ytrain, ytest = convert_dataset_for_models(df=part, seed=seed)
        knn_params = {
            # "n_neighbors": np.arange(1, 100, 10),
            "n_neighbors": [1] + [n for n in range(10,101,10)],
            # "random_state" : random_states,
        }
        knn = KNeighborsClassifier()
        knn_grid = GridSearchCV(estimator = knn, param_grid = knn_params, cv=5, return_train_score = True, scoring='accuracy')
        knn_grid.fit(xtrain, ytrain)    
        knn_tuning_results[daterange] = dict(knn_grid.best_params_) 
        knn_tuning_results[daterange]["score"] = knn_grid.best_score_
        print("knn: ", ", ".join([f"{key} : {knn_grid.best_params_[key]}" for key in knn_grid.best_params_]))


        dtc_params = {
            # "n_neighbors": np.arange(1, 100, 10),
            "random_state" : random_states,
        }
        dtc = DecisionTreeClassifier()
        dtc_grid = GridSearchCV(estimator = dtc, param_grid = dtc_params, cv=5, return_train_score = True, scoring='accuracy')
        dtc_grid.fit(xtrain, ytrain)    
        dtc_tuning_results[daterange] = dict(dtc_grid.best_params_) 
        dtc_tuning_results[daterange]["score"] = dtc_grid.best_score_
        print("dtc: ", ", ".join([f"{key} : {dtc_grid.best_params_[key]}" for key in dtc_grid.best_params_]))

        rf_params = {
            "n_estimators": [1] + [n for n in range(10,201,20)],
            "random_state" : random_states,
        }

        rf = RandomForestClassifier()
        rf_grid = GridSearchCV(estimator = rf, param_grid = rf_params, cv=5, return_train_score = True, scoring='accuracy')
        rf_grid.fit(xtrain, ytrain)    
        rf_tuning_results[daterange] = dict(rf_grid.best_params_) 
        rf_tuning_results[daterange]["score"] = rf_grid.best_score_
        print("rf: ", ", ".join([f"{key} : {rf_grid.best_params_[key]}" for key in rf_grid.best_params_]))


        nb_params = {
            "var_smoothing": np.logspace(0,-9, num=10)
        }
        nb = GaussianNB()
        nb_grid = GridSearchCV(estimator = nb, param_grid = nb_params, cv=5, return_train_score = True, scoring='accuracy')
        nb_grid.fit(xtrain, ytrain)    
        nb_tuning_results[daterange] = dict(nb_grid.best_params_) 
        nb_tuning_results[daterange]["score"] = nb_grid.best_score_
        print("nb: ", ", ".join([f"{key} : {nb_grid.best_params_[key]}" for key in nb_grid.best_params_]))

        svm_params = {
            'C': np.arange(8, 12) * 0.1,
            'gamma': np.arange(1, 5) * 0.1,
        }
        svm = SVC()
        svm_grid = GridSearchCV(estimator = svm, param_grid = svm_params, cv=5, return_train_score = True, scoring='accuracy')
        svm_grid.fit(xtrain, ytrain)    
        svm_tuning_results[daterange] = dict(svm_grid.best_params_) 
        svm_tuning_results[daterange]["score"] = abs(svm_grid.best_score_)
        print("svm: ", ", ".join([f"{key} : {svm_grid.best_params_[key]}" for key in svm_grid.best_params_]))


    return knn_tuning_results, dtc_tuning_results, rf_tuning_results, nb_tuning_results, svm_tuning_results
    

knn_tuning_results, dtc_tuning_results, rf_tuning_results, nb_tuning_results, svm_tuning_results = tune_winner_models()

DataFrame.from_dict(knn_tuning_results, orient="index").to_pickle("cache/knn_tuning_results.pkl")
DataFrame.from_dict(dtc_tuning_results, orient="index").to_pickle("cache/dtc_tuning_results.pkl")
DataFrame.from_dict(rf_tuning_results, orient="index").to_pickle("cache/rf_tuning_results.pkl")
DataFrame.from_dict(nb_tuning_results, orient="index").to_pickle("cache/nb_tuning_results.pkl")
DataFrame.from_dict(svm_tuning_results, orient="index").to_pickle("cache/svm_tuning_results.pkl")


2021-10-28 -> 2022-04-26
knn:  n_neighbors : 100
dtc:  random_state : 30
rf:  n_estimators : 190, random_state : 2
nb:  var_smoothing : 1.0
svm:  C : 0.8, gamma : 0.1
2021-05-01 -> 2021-10-28
knn:  n_neighbors : 60
dtc:  random_state : 1
rf:  n_estimators : 190, random_state : 1
nb:  var_smoothing : 1.0
svm:  C : 0.9, gamma : 0.1
2020-11-02 -> 2021-05-01
knn:  n_neighbors : 60
dtc:  random_state : 30
rf:  n_estimators : 190, random_state : 1
nb:  var_smoothing : 1.0
svm:  C : 0.8, gamma : 0.1
2020-05-06 -> 2020-11-02
knn:  n_neighbors : 90
dtc:  random_state : 2
rf:  n_estimators : 70, random_state : 1
nb:  var_smoothing : 1.0
svm:  C : 0.8, gamma : 0.1
2019-11-08 -> 2020-05-06
knn:  n_neighbors : 100
dtc:  random_state : 6
rf:  n_estimators : 190, random_state : 2
nb:  var_smoothing : 1.0
svm:  C : 1.1, gamma : 0.1
2019-05-12 -> 2019-11-08
knn:  n_neighbors : 100
dtc:  random_state : 6
rf:  n_estimators : 90, random_state : 260
nb:  var_smoothing : 1.0
svm:  C : 0.9, gamma : 0.1
2018-

In [37]:
knn_tuning_results_df : DataFrame = pd.read_pickle("cache/knn_tuning_results.pkl")
dtc_tuning_results_df : DataFrame = pd.read_pickle("cache/dtc_tuning_results.pkl")
rf_tuning_results_df : DataFrame = pd.read_pickle("cache/rf_tuning_results.pkl")
nb_tuning_results_df : DataFrame = pd.read_pickle("cache/nb_tuning_results.pkl")
svm_tuning_results_df : DataFrame = pd.read_pickle("cache/svm_tuning_results.pkl")

models_best_results = pd.DataFrame(knn_tuning_results_df["score"].rename("knn_score"))
models_best_results["dtc_score"] = dtc_tuning_results_df["score"]
models_best_results["rf_score"] = rf_tuning_results_df["score"]
models_best_results["nb_score"] = nb_tuning_results_df["score"]
models_best_results["svm_score"] = svm_tuning_results_df["score"]
models_best_results

models_best_results_stats = pd.DataFrame(knn_tuning_results_df["score"].rename("knn_score").describe())
models_best_results_stats["dtc_score"] = dtc_tuning_results_df["score"].describe()
models_best_results_stats["rf_score"] = rf_tuning_results_df["score"].describe()
models_best_results_stats["nb_score"] = nb_tuning_results_df["score"].describe()
models_best_results_stats["svm_score"] = svm_tuning_results_df["score"].describe()
models_best_results_stats

,knn_score,dtc_score,rf_score,nb_score,svm_score
count,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.632614,0.594361,0.623172,0.616921,0.620508
std,0.012266,0.013257,0.016191,0.024442,0.014989
min,0.609182,0.573623,0.597506,0.583207,0.594590
25%,0.628665,0.589831,0.612768,0.601957,0.609397
50%,0.634208,0.591872,0.616784,0.606949,0.624569
75%,0.639838,0.605052,0.634668,0.638503,0.631872
max,0.650539,0.609722,0.647116,0.650297,0.638484




### Feature Importances

In [ ]:
# fig = plt.figure(figsize=(16, 9))
# sorted_feat = rf.feature_importances_.argsort()
# plt.barh(xtrain.columns,rf.feature_importances_[sorted_feat])

# plt.show()

In [ ]:
# fig = plt.figure(figsize=(16, 9))

# plot_tree(dtc, 
#           feature_names=xtrain.columns,
#           class_names=["t1", "t2"], 
#           filled=True, impurity=True, 
#           rounded=True)

# plt.show()

-----
-----

# ROUND WIN DIFF PREDICTOR MODELS

-----

In [ ]:
def convert_dataset_for_reg_models(df : DataFrame, seed : int = 1, test_set_size : float = 0.3):
    
    teams_profiles : dict[str,Series] = make_teams_profiles(df, pstat_vars=SELECTED_PSTAT_VARS)

    # convert all unranked teams to have rank of 31st position
    model_dataset = df.copy()
    model_dataset["t1_rank"].fillna(31, inplace=True)
    model_dataset["t2_rank"].fillna(31, inplace=True)

    # extract data for each team, extract player stats later
    teams_stats = model_dataset.loc[:,["winner","t1_rank", "t2_rank", "t1_name", "t2_name"]]
    # keep_dict = {}
    # print(teams_stats.shape)
    # add team player stat profiles to each entry / row of data in features dataframe
    features_dict = {}
    for index, row in teams_stats.iterrows():
        t1_name, t2_name = row["t1_name"], row["t2_name"]
        if t1_name in teams_profiles or t2_name in teams_profiles:
            if not t1_name in teams_profiles:
                t2_profile = Series(list(teams_profiles[t2_name].values), 
                    index = [f"t2{pstat_var}" for pstat_var in list(teams_profiles[t2_name].index)])
                t1_profile = Series(list(teams_profiles[t2_name].apply(lambda x: None).values), 
                    index = [f"t1{pstat_var}" for pstat_var in list(teams_profiles[t2_name].index)])
            elif not t2_name in teams_profiles:
                t1_profile = Series(list(teams_profiles[t1_name].values), 
                    index = [f"t1{pstat_var}" for pstat_var in list(teams_profiles[t1_name].index)])
                t2_profile = Series(list(teams_profiles[t1_name].apply(lambda x: None).values), 
                    index = [f"t2{pstat_var}" for pstat_var in list(teams_profiles[t1_name].index)])
            else:
                t1_profile = Series(list(teams_profiles[t1_name].values), 
                    index = [f"t1{pstat_var}" for pstat_var in list(teams_profiles[t1_name].index)])
                t2_profile = Series(list(teams_profiles[t2_name].values), 
                    index = [f"t2{pstat_var}" for pstat_var in list(teams_profiles[t2_name].index)])
            features_dict[index] = pd.concat([row, t1_profile, t2_profile])
            # keep_dict[index] = row
        # else:
            # model_dataset.drop(index, inplace=True)
    
    features = DataFrame.from_dict(features_dict, orient="index")
    features.drop(columns=["t1_name", "t2_name"], inplace=True)
    features["winner"] = features["winner"].apply(lambda x: x == "t1")
    # print(features["winner"])
    # print(list(features.columns))

    features.fillna(0,inplace=True)

    # numerical_vars = [var for var in list(features.columns) if var[0] == "t" and var[2] == "p"]
    numerical_vars = list(features.columns)
    
    labels = model_dataset["rw_diff"].astype("int64")
    # print(features.shape, labels.shape)
    xtrain, xtest, ytrain, ytest =  train_test_split( features, labels, random_state=seed, test_size=test_set_size)

    scaler = StandardScaler()
    scaler.fit(xtrain[numerical_vars])

    xtrain[numerical_vars] = scaler.transform(xtrain[numerical_vars]) #scale the training data
    xtest[numerical_vars] = scaler.transform(xtest[numerical_vars]) #scale the testing data

    return xtrain, xtest, ytrain, ytest


xtrain, xtest, ytrain, ytest = convert_dataset_for_reg_models(recent_matches_dataset ,seed=69)
xtrain.head()


## Initial Regressor Testing/Proof-of-Concept

In [ ]:
def initial_test_rw_diff_models():
    xtrain, xtest, ytrain, ytest = convert_dataset_for_reg_models(dataset_partitions.iloc[1] ,seed=69)
    # print(list(xtrain.columns))

    knn = KNeighborsRegressor(n_neighbors=15)
    knn.fit(xtrain,ytrain)
    knn_ypred = knn.predict(xtest)
    print(list(knn_ypred)[:10])
    print(list(ytest)[:10])
    print(f'The KNeighborsRegressor has a RMSE of {round(mean_squared_error(ytest, knn_ypred, squared=False),2)}')
    print(f'The KNeighborsRegressor has a MAE of {round(mean_absolute_error(ytest, knn_ypred),2)}')


    dtc = DecisionTreeRegressor(random_state=69)
    dtc.fit(xtrain, ytrain)
    dtc_ypred = dtc.predict(xtest)
    print(f'The DecisionTreeRegressor has a RMSE of {round(mean_squared_error(ytest, dtc_ypred, squared=False),2)}')
    print(f'The DecisionTreeRegressor has a MAE of {round(mean_absolute_error(ytest, dtc_ypred),2)}')


    rf = RandomForestRegressor(criterion='squared_error', n_estimators = 100, random_state = 69)
    rf.fit(xtrain, ytrain)
    rf_ypred   = rf.predict(xtest)
    print(f'The RandomForestRegressor has a RMSE of {round(mean_squared_error(ytest, rf_ypred, squared=False),2)}')
    print(f'The RandomForestRegressor has a MAE of {round(mean_absolute_error(ytest, rf_ypred),2)}')

    svm = SVR()
    svm.fit(xtrain, ytrain)
    svm_ypred = svm.predict(xtest)
    print(f'The SVR has a RMSE of {round(mean_squared_error(ytest, svm_ypred, squared=False),2)}')
    print(f'The SVR has a MAE of {round(mean_absolute_error(ytest, svm_ypred),2)}')

initial_test_rw_diff_models()

In [ ]:
dummy_knn = KNeighborsRegressor()
print("Available params for knn():")
print(", ".join([f"{key}: {dummy_knn.get_params()[key]}" for key in dummy_knn.get_params()]))

dummy_dtc = DecisionTreeRegressor()
print("\nAvailable params for dtc():")
print(", ".join([f"{key}: {dummy_dtc.get_params()[key]}" for key in dummy_dtc.get_params()]))

dummy_rf = RandomForestRegressor()
print("\nAvailable params for rf():")
print(", ".join([f"{key}: {dummy_rf.get_params()[key]}" for key in dummy_rf.get_params()]))

dummy_svm = SVR()
print("\nAvailable params for svr():")
print(", ".join([f"{key}: {dummy_svm.get_params()[key]}" for key in dummy_svm.get_params()]))




In [ ]:
def tune_rw_diff_models():
    knn_reg_tuning_results = {}
    dtc_reg_tuning_results =  {}
    rf_reg_tuning_results = {}
    svm_reg_tuning_results = {}

    for daterange, part in dataset_partitions.iteritems():
        print(daterange)
        # 10 different seeds for us to average results across
        # random_states = [6, 13, 23, 24, 33, 69, 420, 666, 6969, 13666]
        # random_states = [69, 420, 666]
        random_states = [n + n**n for n in range(0,5)]
        # knn_tuning_results_dict =  {}
        xtrain, xtest, ytrain, ytest = convert_dataset_for_reg_models(df=part, seed=1)

        # xtrain, xtest, ytrain, ytest = convert_dataset_for_models(df=part, seed=seed)
        knn_params = {
            # "n_neighbors": np.arange(1, 100, 10),
            "n_neighbors": [1] + [n for n in range(10,101,10)],
            # "random_state" : random_states,
        }
        knn = KNeighborsRegressor()        
        knn_grid = GridSearchCV(estimator = knn, param_grid = knn_params, cv=5, return_train_score = True, scoring='neg_mean_absolute_error')
        knn_grid.fit(xtrain, ytrain)    
        knn_reg_tuning_results[daterange] = dict(knn_grid.best_params_) 
        # take absolute value bc cvgridsearch uses negative values for optimization
        knn_reg_tuning_results[daterange]["score"] = abs(knn_grid.best_score_)
        
        print("knn: ", ", ".join([f"{key} : {knn_grid.best_params_[key]}" for key in knn_grid.best_params_]))


        dtc_params = {
            # "n_neighbors": np.arange(1, 100, 10),
            "random_state" : random_states,
        }
        dtc = DecisionTreeRegressor()
        dtc_grid = GridSearchCV(estimator = dtc, param_grid = dtc_params, cv=5, return_train_score = True, scoring='neg_mean_absolute_error')
        dtc_grid.fit(xtrain, ytrain)    
        dtc_reg_tuning_results[daterange] = dict(dtc_grid.best_params_) 
        # take absolute value bc cvgridsearch uses negative values for optimization
        dtc_reg_tuning_results[daterange]["score"] = abs(dtc_grid.best_score_)

        print("dtc: ", ", ".join([f"{key} : {dtc_grid.best_params_[key]}" for key in dtc_grid.best_params_]))

        rf_params = {
            "n_estimators": [1] + [n for n in range(20,201,20)],
            "random_state" : random_states,
        }
        rf = RandomForestRegressor()
        rf_grid = GridSearchCV(estimator = rf, param_grid = rf_params, cv=5, return_train_score = True, scoring='neg_mean_absolute_error')
        rf_grid.fit(xtrain, ytrain)    
        rf_reg_tuning_results[daterange] = dict(rf_grid.best_params_) 
        # take absolute value bc cvgridsearch uses negative values for optimization
        rf_reg_tuning_results[daterange]["score"] = abs(rf_grid.best_score_)

        print("rf: ", ", ".join([f"{key} : {rf_grid.best_params_[key]}" for key in rf_grid.best_params_]))
            
        svm_params = {
            'C': np.arange(8, 12) * 0.1,
            'gamma': np.arange(1, 5) * 0.1,
        }
        svm = SVR()
        svm_grid = GridSearchCV(estimator = svm, param_grid = svm_params, cv=5, return_train_score = True, scoring='neg_mean_absolute_error')
        svm_grid.fit(xtrain, ytrain)    
        svm_reg_tuning_results[daterange] = dict(svm_grid.best_params_) 
        # take absolute value bc cvgridsearch uses negative values for optimization
        svm_reg_tuning_results[daterange]["score"] = abs(svm_grid.best_score_)
        print("svm: ", ", ".join([f"{key} : {svm_grid.best_params_[key]}" for key in svm_grid.best_params_]))


    return knn_reg_tuning_results, dtc_reg_tuning_results, rf_reg_tuning_results, svm_reg_tuning_results
knn_reg_tuning_results, dtc_reg_tuning_results, rf_reg_tuning_results, svm_reg_tuning_results = tune_rw_diff_models()

DataFrame.from_dict(knn_reg_tuning_results, orient="index").to_pickle("cache/knn_reg_tuning_results.pkl")
DataFrame.from_dict(dtc_reg_tuning_results, orient="index").to_pickle("cache/dtc_reg_tuning_results.pkl")
DataFrame.from_dict(rf_reg_tuning_results, orient="index").to_pickle("cache/rf_reg_tuning_results.pkl")
DataFrame.from_dict(svm_reg_tuning_results, orient="index").to_pickle("cache/svm_reg_tuning_results.pkl")


In [ ]:

# knn_reg_tuning_results_df = pd.read_pickle("cache/knn_reg_tuning_results.pkl")
# dtc_reg_tuning_results_df = DataFrame.from_dict(dtc_reg_tuning_results, orient="index")
# rf_reg_tuning_results_df = DataFrame.from_dict(rf_reg_tuning_results, orient="index")
svm_reg_tuning_results_df = pd.read_pickle("cache/svm_reg_tuning_results.pkl")

# print("KNN Regression Tuning Results:")
# print(knn_reg_tuning_results_df)
# print("DTC Regression Tuning Results:")
# print(dtc_reg_tuning_results_df)
# print("RF Regression Tuning Results:")
# print(rf_reg_tuning_results_df)
print("SVM Regression Tuning Results:")
print(svm_reg_tuning_results_df)

In [ ]:
random_states = [n + n**n for n in range(0,10)]
random_states